In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,SimpleRNN 
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
import keras.utils as utils
import re
from keras.callbacks import ModelCheckpoint
import math
import codecs
import random
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from nltk import ngrams

# Classical **Apprroach**

In [5]:
f = open('speeches.txt', 'r', encoding='UTF-8')
raw = f.read()
raw = raw.replace("\'", "")
raw = raw.replace(",", "")
raw = raw.replace("\r\n", "")
raw = raw.replace("\ufeff","")
raw = re.sub(r'SPEECH \d',r'',raw)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
special_chars = re.compile('[`~!@#$%^&*()+={}|\[\]:";<>?,\./“”]')
raw_sentence = sent_tokenize(raw)
sentences = []
for item in raw_sentence:
  text = item.replace("\n", " ")
  text = "<s> " + special_chars.sub("", text) + " </s>"
  text = text.lower()
  sentences.append(' '.join(text.split()))
  


In [0]:
xtrain, xtest = train_test_split(sentences, test_size = 0.2, random_state = 0)

In [0]:
def retngrams(xtrain,n):
  dictk=dict()
  count = 0
  for sen in xtrain:
    kgrams = ngrams(sen.split(), n)
    for i in kgrams:
      if(i not in dictk):
        dictk[i]=1
      else:
        dictk[i]=dictk[i]+1
      count=count+1
  return(dictk,count)
  

In [9]:
igram = []
for i in range(1,5):
  igram.append(retngrams(xtrain,i))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [0]:
#@title MLE


In [0]:
MLE = []
for i in range(4):
  d = dict()
  if(i == 0):
    for j in igram[i][0]:
      d[j] = igram[i][0][j]/igram[i][1]
    MLE.append(d)
  else:
    for j in igram[i][0]:
      d[j] = igram[i][0][j]/igram[i-1][0][j[:-1]]
    MLE.append(d)

In [11]:
sum(MLE[1].values())

5900.999999999846

In [0]:
def Generator(n,size, igram):
  ans = "<s>"
  sentence = ["<s>"]
  if(n == 1):
    keys = list(MLE[n-1].keys())
    values = list(MLE[n-1].values())
    rand = np.random.multinomial(1 , values , size=size)
    for i in rand:
      for j in range(len(keys)):
        if(i[j] == 1):
          if(keys[j][0] == "<s>"):
            sentence.append(keys[j][0])
            print(ans)
            return
          sentence.append(keys[j][0])
          ans = ans + " " + keys[j][0]
          if(keys[j][0] == "</s>"):
            print(ans)
            return
            break
    print(ans)
    return       
  else:
    sentence=['<s>']
    tempa=[]
    tempb=[]
    for i in range(n-1):
      sentence=list(sentence)
      keys = list(igram[i].keys())
      values = list(igram[i].values())
      for key in keys:
        ls=list(key)
        if(ls[:i]==sentence):
          tempa.append(key)
          tempb.append(igram[i][key])
        rand=np.random.multinomial(1 , tempb , size=1)
        for k in range(len(rand[0])):
          if(rand[0][k]==1):
            sentence=tempa[k]
        tempa = []
        tempb = []
    sentence=list(sentence)
    tempa=[]
    tempb=[]
    keys = list(igram[n-1].keys())
    values = list(igram[n-1].values())
    print(*sentence,end=' ')
    for i in range(size):
      for key in keys:
        ls=list(key)
          if(ls[:-1]==sentence):
            tempa.append(key)
            tempb.append(igram[n-1][key])
        rand=np.random.multinomial(1 , tempb , size=1)
        for k in range(len(rand[0])):
          if(rand[0][k]==1):
            print(list(tempa[k])[-1],end=' ')
            sentence=list(tempa[k])[1:]
        setence=list(sentence)
        tempa=[]
        tempb=[]    
    

# Sentence Generation and Perplexity **calculations**

In [29]:
def probab(sentence, n):
  prob = 0
  sentence = sentence.split(" ")
  for i in range(len(sentence)-n+1):
    w = sentence[i:i+n]
    prob += math.log(MLE[n-1][tuple(w)],2)
  return math.pow(2,prob)

def perplexity(tests, n):
  prob = 0
  len1 = 0
  for i in tests:
    try:
      prob -= math.log(probab(i, n),2)
    except:
      prob -= 0
    sentence = i.split(" ")
    len1 += len(sentence)-n-1
  return math.pow(2, prob/len1)

print("Unigram perplexity : ",perplexity(xtest, 1))
print("Randomly generated Sentence from Unigram : " )
print(Generator(1,30,MLE))
print(" ")
print("Bigram perplexity : ",perplexity(xtest, 2))
print("Randomly generated Sentence from Bigram : " )
Generator(2,30,MLE)
print(" ")
print(" ")
print("Trigram perplexity : ",perplexity(xtest, 3))
print("Randomly generated Sentence from Trigram : " )
Generator(3,30,MLE)
print(" ")
print(" ")
print("Quadgram perplexity : ",perplexity(xtest, 4))

print("Randomly generated Sentence from Quadgram : " )
print(Generator(4,30,MLE))

Unigram perplexity :  222.83338830403525
Randomly generated Sentence from Unigram : 
<s> this ankles ford that’s certain to badly room asked with all i’m many
None
 
Bigram perplexity :  2.3085702897936864
Randomly generated Sentence from Bigram : 
<s> i intend to have the other laws are going up his friends were fighting everybody </s>  
 
Trigram perplexity :  1.2051141445849718
Randomly generated Sentence from Trigram : 
<s> both of them moving into mexico </s>  
 
Quadgram perplexity :  1.0634054681063243
Randomly generated Sentence from Quadgram : 
<s> first i said oh this is a common sense well thought out tax proposal that’s going to be negotiating our deals and they’re the — they’re the easiest </s> None


# Neural **Networks**

In [0]:
neural = []
for i in xtrain:
  neural.append(i.split())
tokenizer = Tokenizer()
tokenizer.fit_on_texts(neural)
len_total= len(tokenizer.word_index) + 1
train_neural = []
for i in xtrain:
  tokens = tokenizer.texts_to_sequences([i])[0]
  for j in range(1, len(tokens)):
        train_neural.append(tokens[:j+1])
train_neural = np.array(pad_sequences(train_neural,maxlen=15, padding='pre'))
predictors, label = train_neural[:,:-1],train_neural[:,-1]
label = utils.to_categorical(label, num_classes=len_total)


In [56]:
test = retngrams(xtest,4)
n_test = []
for i in test[0].keys():
  a = []
  for j in i :
    a.append(j)
  n_test.append(a)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [105]:
from keras.layers import SimpleRNN
model_rnn = Sequential()
model_rnn.add(Embedding(len_total, 100, input_length=14))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dropout(0.2))
model_rnn.add(Dense(len_total, activation='softmax'))

model_rnn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 14, 100)           590300    
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 100)               20100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 5903)              596203    
Total params: 1,206,603
Trainable params: 1,206,603
Non-trainable params: 0
_________________________________________________________________


In [104]:
model_rnn.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model_rnn.fit(predictors,label, batch_size=128, epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
122058/122058 [==============================] - 45s 367us/step - loss: 6.1897 - acc: 0.0540
Epoch 2/10
122058/122058 [==============================] - 43s 356us/step - loss: 5.6844 - acc: 0.0999
Epoch 3/10
122058/122058 [==============================] - 43s 356us/step - loss: 5.4161 - acc: 0.1264
Epoch 4/10
122058/122058 [==============================] - 43s 355us/step - loss: 5.2918 - acc: 0.1366
Epoch 5/10
122058/122058 [==============================] - 44s 357us/step - loss: 5.2202 - acc: 0.1445
Epoch 6/10
122058/122058 [==============================] - 44s 363us/step - loss: 5.1697 - acc: 0.1499
Epoch 7/10
122058/122058 [==============================] - 45s 368us/step - loss: 5.1342 - acc: 0.1543
Epoch 8/10
122058/122058 [==============================] - 45s 370us/step - loss: 5.1098 - acc: 0.1580
Epoch 9/10
122058/122058 [==============================] - 46s 378us/step

In [0]:
def Generator(text,model):
  for j in range(15):
    tokens = tokenizer.texts_to_sequences([text])[0]
    tokens = pad_sequences([tokens], maxlen= 14, padding='pre')
    pred = model.predict_classes(tokens, verbose=0)
    temp = ""
    for tex, i in tokenizer.word_index.items():
      if i == pred:
        text += " " + tex
        break
  return(text)

# **Sentence Generation for RNN**

In [129]:
print(Generator("am",model_rnn))
print(Generator("i",model_rnn))
print(Generator("america",model_rnn))

am table table heros struggles heros gps takes bone changer excused killings prove dollar lost soul
i table cool table ex-president 5 degenerates bit lakes dream bosses racial mountain resolutions america intelligence
america table table 5 anti-ship vladimir gps better tools bit stamped heros handled many-- bless doesnt


In [0]:
model_lstm = Sequential()
model_lstm.add(Embedding(len_total, 50, input_length=14))
model_lstm.add(LSTM(100))
model.add(Dropout(0.2))

model_lstm.add(Dense(40, activation='relu'))
model_lstm.add(Dense(len_total, activation='softmax'))
model_lstm.summary()

In [0]:
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(predictors,label, batch_size=128, epochs=10)

Epoch 1/10
122058/122058 [==============================] - 77s 632us/step - loss: 6.2339 - acc: 0.0545
Epoch 2/10
122058/122058 [==============================] - 77s 628us/step - loss: 5.6262 - acc: 0.1135
Epoch 3/10
122058/122058 [==============================] - 76s 620us/step - loss: 5.2599 - acc: 0.1452
Epoch 4/10
122058/122058 [==============================] - 76s 619us/step - loss: 5.0109 - acc: 0.1640
Epoch 5/10
122058/122058 [==============================] - 77s 627us/step - loss: 4.8232 - acc: 0.1767
Epoch 6/10
122058/122058 [==============================] - 78s 636us/step - loss: 4.6744 - acc: 0.1872
Epoch 7/10
122058/122058 [==============================] - 76s 624us/step - loss: 4.5457 - acc: 0.1963
Epoch 8/10
122058/122058 [==============================] - 77s 627us/step - loss: 4.4308 - acc: 0.2036
Epoch 9/10
104832/122058 [========================>.....] - ETA: 10s - loss: 4.3286 - acc: 0.2119

In [0]:
def perplexity(y_true, y_pred):
    cross_entropy = Keras.sparse_categorical_crossentropy(y_true, y_pred)
    perplexity = Keras.exp(cross_entropy)
    return perplexity

# `Sentence Generation for **LSTM**`

In [137]:
print(Generator("am",model_lstm))
print(Generator("i",model_lstm))
print(Generator("trump",model_lstm))

am the way i said i said i said i said i said i said i
i said i said i said i said i said i said i said i said
trump is a lot of money and i said i said i said i said i


By the looks of it, classical performs better, it is able to generate better sentencse which are grammatically correct and make sense as well, LSTM performs poor as of now, it may perform better if we tune it better. 